In [48]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from matplotlib.lines import Line2D
from scipy.integrate import trapezoid


In [42]:
q   = 100.0             # [Вт] Мощность источника
k   = 28.0              # [Вт/(м·K)] Теплопроводность
rho = 4400.0            # [кг/м^3] Плотность
c   = 760.0             # [Дж/(кг·K)] Теплоёмкость
v   = 1.0               # [м/с] Скорость
a   = k/(rho*c)         # [м^2/с] Теплодиффузивность
r0  = 40e-6             # [м] Характерный масштаб (40 мкм)
t0 = r0**2 / (4 * a)    # Характерное время
V = v**2 * t0 / r0**2   # Безразмерная скорость


In [43]:
R = lambda x, y, z: np.sqrt(x**2 + y**2 + z**2)
r = lambda x, y: np.sqrt(x**2 + y**2)
H = lambda x, y: (r(x, y)/r0)**2
Z = lambda z: (z/r0)**2
T_ideal = lambda x, y, z: q/(2*np.pi*k*R(x,y,z))*np.exp(-v/(2*a)*(x+R(x,y,z)))

In [78]:
def phi1(x, y, z, lam):
    H_ = H(x, y)
    Z_ = Z(z)
    sin = np.sin(lam)**2
    cos = np.cos(lam)**2
    ph = np.exp(-Z_/sin - H_*cos - V/cos)
    return ph

def phi2(x, y, z, lam):
    H_ = H(x, y)
    Z_ = Z(z)
    sin = np.sin(lam)**2
    cos = np.cos(lam)**2
    ph = np.exp(-Z_/cos + H_*cos - V/sin)
    return ph

def I(x, y, z, N):
    H_ = H(x, y)
    Z_ = Z(z)
    lam = np.linspace(1e-9, np.pi/4, N)  # избегаем деления на 0 в 0
    phi1_vals = phi1(x, y, z, lam)
    phi2_vals = phi2(x, y, z, lam)

    I1 = trapezoid(phi1_vals, lam)
    I2 = trapezoid(phi2_vals, lam)


    i = np.exp(Z_) * (I1 + np.exp(-H_) * I2)
    return i

def T_real(x, y, z, N):
    pre = 2*q/(rho*c*(4*np.pi*a)**1.5) 
    T = pre * np.exp(-v*x/(2*a)) * I(x, y, z, N)
    return T


Z(0): 0.0
H(0,0): 0.0
V: 29857.14285714286
pre: 55.41295202119376
T_real(0,0,0,N): 0.0


In [71]:
# Изотермы плавления и испарения (в К) для двух сплавов
Ti_iso = {
    "1920 K (Ti)": 1920,
    "3530 K (Ti)": 3530
}
# Al_iso = {
#     "870 K (Al)": 870,
#     "2600 K (Al)": 2600
# }

# Цвета
Ti_color = 'cyan'
Al_color = 'magenta'

# Границы и сетка
L = 1e-3
N = 200
grid = np.linspace(-L, L, N)

# Ползунки
slider_min = 0.0
slider_max = 400e-6
slider_step = 1e-6

def add_isotherms(ax, X, Y, T):
    # Ti
    cs1 = ax.contour(X, Y, T, levels=Ti_iso.values(), colors=Ti_color, linewidths=2)
    ax.clabel(cs1, fmt={v: k for k, v in Ti_iso.items()}, colors=Ti_color)

    # Al
    cs2 = ax.contour(X, Y, T, levels=Al_iso.values(), colors=Al_color, linewidths=2)
    ax.clabel(cs2, fmt={v: k for k, v in Al_iso.items()}, colors=Al_color)

    # Легенда
    proxy_lines = [
        Line2D([0], [0], color=Ti_color, lw=2, label='Ti-6Al-4V'),
        Line2D([0], [0], color=Al_color, lw=2, label='AlSi10Mg')
    ]
    ax.legend(handles=proxy_lines, loc='upper right')

In [77]:
@interact(z=FloatSlider(min=slider_min, max=slider_max, step=slider_step, value=r0, description='fix(z)'))
def plot_xy(z):
    X, Y = np.meshgrid(grid, grid)
    T = T_ideal(X, Y, z)
    fig, ax = plt.subplots(figsize=(8, 6))
    cf = ax.contourf(X * 1e6, Y * 1e6, T, levels=50, cmap="hot")
    fig.colorbar(cf, label='T, K')
    add_isotherms(ax, X * 1e6, Y * 1e6, T)
    ax.set_xlabel('x, мкм')
    ax.set_ylabel('y, мкм')
    ax.set_title(f'T_xy при z = {z*1e6:.0f} мкм')
    plt.show()

@interact(y=FloatSlider(min=slider_min, max=slider_max, step=slider_step, value=r0, description='fix(y)'))
def plot_xz(y):
    X, Z = np.meshgrid(grid, grid)
    T = T_ideal(X, y, Z)
    fig, ax = plt.subplots(figsize=(8, 6))
    cf = ax.contourf(X * 1e6, Z * 1e6, T, levels=50, cmap="hot")
    fig.colorbar(cf, label='T, K')
    add_isotherms(ax, X * 1e6, Z * 1e6, T)
    ax.set_xlabel('x, мкм')
    ax.set_ylabel('z, мкм')
    ax.set_title(f'T_xz при y = {y*1e6:.0f} мкм')
    plt.show()

@interact(x=FloatSlider(min=slider_min, max=slider_max, step=slider_step, value=r0, description='fix(x)'))
def plot_yz(x):
    Y, Z = np.meshgrid(grid, grid)
    T = T_ideal(x, Y, Z)
    fig, ax = plt.subplots(figsize=(8, 6))
    cf = ax.contourf(Y * 1e6, Z * 1e6, T, levels=50, cmap="hot")
    fig.colorbar(cf, label='T, K')
    add_isotherms(ax, Y * 1e6, Z * 1e6, T)
    ax.set_xlabel('y, мкм')
    ax.set_ylabel('z, мкм')
    ax.set_title(f'T_yz при x = {x*1e6:.0f} мкм')
    plt.show()


interactive(children=(FloatSlider(value=4e-05, description='fix(z)', max=0.0004, step=1e-06), Output()), _dom_…

interactive(children=(FloatSlider(value=4e-05, description='fix(y)', max=0.0004, step=1e-06), Output()), _dom_…

interactive(children=(FloatSlider(value=4e-05, description='fix(x)', max=0.0004, step=1e-06), Output()), _dom_…

In [73]:
@interact(z=FloatSlider(min=slider_min, max=slider_max, step=slider_step, value=0.0, description='fix(z)'))
def plot_xy(z):
    X, Y = np.meshgrid(grid, grid)
    T = np.vectorize(T_real)(X, Y, z, N)
    fig, ax = plt.subplots(figsize=(8, 6))
    cf = ax.contourf(X * 1e6, Y * 1e6, T, levels=50, cmap="hot")
    fig.colorbar(cf, label='T, K')
    add_isotherms(ax, X * 1e6, Y * 1e6, T)
    ax.set_xlabel('x, мкм')
    ax.set_ylabel('y, мкм')
    ax.set_title(f'T_xy при z = {z*1e6:.0f} мкм')
    plt.show()

@interact(y=FloatSlider(min=slider_min, max=slider_max, step=slider_step, value=0.0, description='fix(y)'))
def plot_xz(y):
    X, Z = np.meshgrid(grid, grid)
    T = np.vectorize(T_real)(X, y, Z, N)
    fig, ax = plt.subplots(figsize=(8, 6))
    cf = ax.contourf(X * 1e6, Z * 1e6, T, levels=50, cmap="hot")
    fig.colorbar(cf, label='T, K')
    add_isotherms(ax, X * 1e6, Z * 1e6, T)
    ax.set_xlabel('x, мкм')
    ax.set_ylabel('z, мкм')
    ax.set_title(f'T_xz при y = {y*1e6:.0f} мкм')
    plt.show()

@interact(x=FloatSlider(min=slider_min, max=slider_max, step=slider_step, value=0.0, description='fix(x)'))
def plot_yz(x):
    Y, Z = np.meshgrid(grid, grid)
    T = np.vectorize(T_real)(x, Y, Z, N)
    fig, ax = plt.subplots(figsize=(8, 6))
    cf = ax.contourf(Y * 1e6, Z * 1e6, T, levels=50, cmap="hot")
    fig.colorbar(cf, label='T, K')
    add_isotherms(ax, Y * 1e6, Z * 1e6, T)
    ax.set_xlabel('y, мкм')
    ax.set_ylabel('z, мкм')
    ax.set_title(f'T_yz при x = {x*1e6:.0f} мкм')
    plt.show()


interactive(children=(FloatSlider(value=0.0, description='fix(z)', max=0.0004, step=1e-06), Output()), _dom_cl…

interactive(children=(FloatSlider(value=0.0, description='fix(y)', max=0.0004, step=1e-06), Output()), _dom_cl…

interactive(children=(FloatSlider(value=0.0, description='fix(x)', max=0.0004, step=1e-06), Output()), _dom_cl…